# Installating Julia/IJulia on Jupyter-Portable (www.portabledevapps.net) via a Notebook
###1 - Downloading and Installing the right Julia binary in the right place

script downloaded from: https://github.com/winpython/winpython_afterdoc/blob/master/examples/installing_julia_and_ijulia.ipynb

and slightly adapted.

In [ ]:
import os
import sys
import io

Change the URLs, Checksums and JuliaVersion String in the following two cells and execute the cells:

In [ ]:
JuliaVersion='1.3.1'

In [ ]:
# downloading julia (~50MB, may take 1 minute or 2)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    julia_binary="julia-"+JuliaVersion+"-win64.exe"
    julia_url="https://julialang-s3.julialang.org/bin/winnt/x64/1.3/julia-1.3.1-win64.exe"
    hashes=("fe285b4c7b91a038528a45715921e34c", "8350ca66f80484c5ca6f7341ffbdb9d5182f8d4231762d585e229567b227ef7f" )
else:
    julia_binary="julia-"+JuliaVersion+"-win32.exe"
    julia_url="https://julialang-s3.julialang.org/bin/winnt/x86/1.3/julia-1.3.1-win32.exe"
    hashes=("56efbc8bafe2702b28b23f0aaa9840cb", "6f2255d7e1707af00549f06b334d7794c4cde5a1eb92776e31142fdf294768be")
    
julia_installer=os.environ["HOME"]+"\\"+julia_binary
os.environ["julia_installer"]=julia_installer

Run everything below:

In [ ]:
g = urllib2.urlopen(julia_url) 
with io.open(julia_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
#checking it's there
!dir %julia_installer%

In [ ]:
# checking it's the official julia binary
import hashlib
def give_hash(of_file, with_this):
    with io.open(julia_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*30+"SHA-256"+" "*30+"\n"+"-"*32+" "+"-"*65)

print ("%s %s %s" % (give_hash(julia_installer, hashlib.md5) , give_hash(julia_installer, hashlib.sha256),julia_installer))
assert give_hash(julia_installer, hashlib.md5) == hashes[0]
assert give_hash(julia_installer, hashlib.sha256) == hashes[1]

In [ ]:
os.environ["JULIA_HOME"] = os.environ["APPSDIR"]+"\\Julia-"+JuliaVersion+"\\"
os.environ["JULIA_EXE_PATH"] = os.path.join(os.environ["JULIA_HOME"],'bin')
os.environ["JULIA_EXE"]="julia.exe"
os.environ["JULIA"]=os.environ["JULIA_EXE_PATH"]+os.environ["JULIA_EXE"]
# for installation we need this
os.environ["JULIAROOT"]=os.environ["JULIA_HOME"]

In [ ]:
print('JULIA_HOME: '+os.environ["JULIA_HOME"])
print('JULIA_EXE_PATH: '+os.environ["JULIA_EXE_PATH"])
print('JULIA_EXE: '+os.environ["JULIA_EXE"])
print('JULIA: '+os.environ["JULIA"])
print('JULIAROOT: '+os.environ["JULIAROOT"])

In [ ]:
# let's install it (add a  /S before /D if you want silence mode installation)

# If you are "USB life style", or multi-winpython
#   ==> UN-CLICK the OPTION 'CREATE a StartMenuFolder and Shortcut' <== (when it will show up)

!start cmd /C %julia_installer% /D=%JULIAROOT%


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/julia_setup_unclick_all.GIF">

###2 - Initialize Julia , IJulia, and make them link to winpython

In [ ]:
# Writing a julia initial run script, for convenience
bat_text = r"""
@echo off
set PATH=%APPSDIR%\Lib\site-packages\PyQt5;%APPSDIR%\Lib\site-packages\PyQt4;%APPSDIR%\;%APPSDIR%\DLLs;%APPSDIR%\scripts;%APPSDIR%;%APPSDIR%\Julia-JuliaVersion\bin;%PATH%

if  exist "%JULIA_HOME%" goto julia_next
echo --------------------
echo First install Julia in \t\Julia-1.0.0 of winpython
echo suggestion : don't create Julia shortcuts, nor menu, nor desktop icons 
echo (they would create a .julia in your home directory rather than here)
echo When this is done, launch this .bat again 

if not exist "%JULIA_HOME%" echo JULIA_HOME is not set in Environment Variables
if not exist "%JULIA_HOME%" goto julia_end

:julia_next
set SYS_PATH=%PATH%
set PATH=%JULIA_EXE_PATH%;%SYS_PATH%

set JULIA_EXE=julia.exe
set JULIA=%JULIA_EXE_PATH%%JULIA_EXE%
set JULIA_DEPOT_PATH=%APPSDIR%\Julia-JuliaVersion\.julia
set JULIA_HOME=%APPSDIR%\Julia-JuliaVersion\
set JULIA_HISTORY=%APPSDIR%\Julia-JuliaVersion\.julia\logs\repl_history.jl

if not exist "%JULIA_DEPOT_PATH%" mkdir "%JULIA_DEPOT_PATH%"

if not exist "%APPSDIR%\Julia-JuliaVersion\.julia\logs\" mkdir "%APPSDIR%\Julia-JuliaVersion\.julia\logs\"

set private_libdir=bin
if not exist "%JULIA_HOME%lib\julia\sys.ji" ( ^
echo "Preparing Julia for first launch. This may take a while"  
popd && pushd "%cd%" )

echo "julia!"
echo --------------------
echo 1) julia package manager will be initialized
echo 2) the WinRPM package will be added (for sysimg compilation)   -- currently not supported
echo 3) the julia sysimg will be compiled for your system.          -- currently not supported
echo 4) the following packages will be installed:
echo    IJulia, PyCall, PyPlot, Interact, Compose, SymPy
echo. 
echo.
echo NOTE: Windows 7/Windows Server 2012 users also require 
echo       TLS "Easy Fix" update, and Windows Management 
echo       Framework 3.0 or later
echo       "Easy Fix": https://bit.ly/2LNjZir
echo.
pause
rem --------- build sys image ---------------
echo using Pkg;>this_is_temporary.jl
echo Pkg.instantiate();>>this_is_temporary.jl
echo Pkg.update();>>this_is_temporary.jl
rem echo Pkg.add("https://github.com/JuliaPackaging/WinRPM.jl");>>this_is_temporary.jl
rem echo using WinRPM;>>this_is_temporary.jl
rem echo WinRPM.update();>>this_is_temporary.jl
rem echo WinRPM.install("gcc"; yes=true);>>this_is_temporary.jl
rem echo include(joinpath(JULIA_HOME, Base.DATAROOTDIR, "julia", "build_sysimg.jl"));>>this_is_temporary.jl
rem echo build_sysimg(ENV["JULIA_HOME"]*"\\lib\\julia\\sys","native",nothing;force=true);>>this_is_temporary.jl
@echo on
%JULIA% this_is_temporary.jl  
@echo off
rem ------------- install IJulia -------------
echo ENV["JUPYTER"]=ENV["APPSDIR"]*"\\scripts\\jupyter";>this_is_temporary2.jl
echo ENV["PYTHON"]=ENV["APPSDIR"]*"\\python.exe";>>this_is_temporary2.jl
echo using Pkg;>>this_is_temporary2.jl
echo Pkg.add("PyCall");>>this_is_temporary2.jl
echo Pkg.add("IJulia");>>this_is_temporary2.jl
echo Pkg.add("PyPlot");>>this_is_temporary2.jl
echo Pkg.add("Interact");>>this_is_temporary2.jl
rem echo Pkg.add("Compose");>>this_is_temporary2.jl
echo Pkg.add("SymPy");>>this_is_temporary2.jl
rem  ---- init pre-compilation ---
echo using PyCall;>>this_is_temporary2.jl
echo using IJulia;>>this_is_temporary2.jl
echo using PyPlot;>>this_is_temporary2.jl
echo using Interact;>>this_is_temporary2.jl
rem echo using Compose;>>this_is_temporary2.jl
echo using SymPy;>>this_is_temporary2.jl
rem echo Pkg.precompile()>>this_is_temporary2.jl
@echo on
%JULIA% this_is_temporary2.jl  
@echo off
echo.
echo.
echo IJulia installed !
echo.
echo.
pause
:julia_end
rem cmd.exe /k
"""
bat_text = bat_text.replace("Julia-JuliaVersion",'Julia-'+JuliaVersion)
julia_initializer_bat=os.environ["APPSDIR"]+"\\scripts\\initialize_julia_once.bat"
if sys.version_info[0] == 3:
    with io.open(julia_initializer_bat, 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in bat_text.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open(julia_initializer_bat, 'wb'  ) as f:
        for line in bat_text.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )

In [ ]:
# let's initialize Julia and install "IJulia", "PyCall", and "PyPlot" Julia modules with this .bat just created
# may take about 10 minutes (Julia pre-compiles itself and download a lot of things)
!start cmd /C %APPSDIR%\\scripts\\initialize_julia_once.bat

In [ ]:
# the path to kernel.jl is hardcoded in the kernel.json file
# this will cause trouble, if the drive letter of the usb drive changes
# use relative paths instead
# rewrite kernel.json and delete the one created from IJulia.jl Package
kernel_path = os.environ["APPSDIR"]+"\\share\\jupyter\\kernels\\julia-"+JuliaVersion

if not os.path.isdir(kernel_path):
    os.mkdir(kernel_path)
    
os.rename(os.environ["APPSDIR"]+"\\datafiles\\kernels\\julia-"+JuliaVersion[0:3]+"\\logo-32x32.png",kernel_path+"\\logo-32x32.png")
os.rename(os.environ["APPSDIR"]+"\\datafiles\\kernels\\julia-"+JuliaVersion[0:3]+"\\logo-64x64.png",kernel_path+"\\logo-64x64.png")

os.remove(os.environ["APPSDIR"]+"\\datafiles\\kernels\\julia-"+JuliaVersion[0:3]+"\\kernel.json")
os.rmdir(os.environ["APPSDIR"]+"\\datafiles\\kernels\\julia-"+JuliaVersion[0:3])

pkg_crcs = os.listdir(os.environ["JULIA_HOME"]+"\\.julia\\packages\\IJulia")

last_mod_time = 0.0
latest_pkg    = ""
for pkdir in pkg_crcs:
    i_mod_time = os.path.getmtime(os.environ["JULIA_HOME"]+"\\.julia\\packages\\IJulia\\"+pkdir)
    if i_mod_time > last_mod_time:
        last_mod_time = i_mod_time
        latest_pkg = pkdir

kernel_str = r"""{
  "display_name": "Julia-JuliaVersion",
  "argv": [
    "{prefix}\\Julia-JuliaVersion\\bin\\julia.exe",
    "-i",
    "--startup-file=yes",
    "--color=yes",
    "--project=@.",
    "{prefix}\\Julia-JuliaVersion\\.julia\\packages\\IJulia\\latest_pkg_dir\\src\\kernel.jl",
    "{connection_file}"
  ],
  "language": "julia",
  "env": {},
  "interrupt_mode": "message"
}
"""

kernel_str = kernel_str.replace("JuliaVersion",JuliaVersion)
kernel_str = kernel_str.replace("latest_pkg_dir",latest_pkg)

if sys.version_info[0] == 3:
    with io.open(kernel_path+"\\kernel.json", 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in kernel_str.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open(kernel_path+"\\kernel.json", 'wb'  ) as f:
        for line in kernel_str.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )


# add JULIA_DEPOT_PATH env variable 
inp_str = r"""

if not exist "%APPSDIR%\Julia-JuliaVersion\bin" goto julia_bad
set JULIA_DEPOT_PATH=%APPSDIR%\Julia-JuliaVersion\.julia
set JULIA_PKGDIR=%APPSDIR%\Julia-JuliaVersion\.julia
rem set JULIA_EXE=julia.exe
set JULIA_HOME=%APPSDIR%\Julia-JuliaVersion\
set JULIA_HISTORY=%APPSDIR%\Julia-JuliaVersion\.julia\logs\repl_history.jl
:julia_bad

"""

inp_str = inp_str.replace("JuliaVersion",JuliaVersion)

# Read in the file
with open(os.environ["APPSDIR"]+"\\scripts\\env.bat", 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata+inp_str

# the env.bat has the "hidden" file attribute and is therefore not writable
os.system("attrib -h "+os.environ["APPSDIR"]+"\\scripts\\env.bat")

# Write the file out again
with open(os.environ["APPSDIR"]+"\\scripts\\env.bat", 'w') as file:
  file.write(filedata)

# restore the hidden file attribute of env.bat
os.system("attrib +h "+os.environ["APPSDIR"]+"\\scripts\\env.bat")

In [ ]:
# clean up

os.remove(os.environ["APPSDIR"]+"\\scripts\\initialize_julia_once.bat")
os.remove(julia_installer)
os.remove("this_is_temporary.jl")
os.remove("this_is_temporary2.jl")